In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import os
import random
import tensorflow as tf
import cv2 as cv

2022-06-18 20:57:05.340547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-18 20:57:05.340571: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Get images
image = img_to_array(load_img('images/woman.jpg'))
image = cv.resize(image, (400, 400))
image = np.array(image, dtype=float)

In [4]:
X = cv.cvtColor(np.float32(1.0/255*image), cv.COLOR_RGB2LAB)[:,:,0]
Y = cv.cvtColor(np.float32(1.0/255*image), cv.COLOR_RGB2LAB)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [5]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

2022-06-18 20:57:26.790994: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/beto/anaconda3/envs/cnn/lib/python3.9/site-packages/cv2/../../lib64:
2022-06-18 20:57:26.791029: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-18 20:57:26.791055: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Archcraft-PC): /proc/driver/nvidia/version does not exist
2022-06-18 20:57:26.791413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse')

In [7]:
# Train model
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=200)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.5178
Epoch 2/200
1/1 [==============================] - 0s 243ms/step - loss: 0.8228
Epoch 3/200
1/1 [==============================] - 0s 239ms/step - loss: 0.0948
Epoch 4/200
1/1 [==============================] - 0s 233ms/step - loss: 0.0086
Epoch 5/200
1/1 [==============================] - 0s 234ms/step - loss: 0.0091
Epoch 6/200
1/1 [==============================] - 0s 257ms/step - loss: 0.0086
Epoch 7/200
1/1 [==============================] - 0s 261ms/step - loss: 0.0083
Epoch 8/200
1/1 [==============================] - 0s 266ms/step - loss: 0.0181
Epoch 9/200
1/1 [==============================] - 0s 263ms/step - loss: 0.0080
Epoch 10/200
1/1 [==============================] - 0s 280ms/step - loss: 0.0073
Epoch 11/200
1/1 [==============================] - 0s 238ms/step - loss: 0.0073
Epoch 12/200
1/1 [==============================] - 0s 276ms/step - loss: 0.0080
Epoch 13/200
1/1 [======================

In [8]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output = output * 128


1/1 [==============================] - 0s 232ms/step - loss: 0.0024
0.0023664270993322134
1/1 [==============================] - 0s 176ms/step


In [9]:
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]

cv_result = cv.cvtColor(np.float32(cur),  cv.COLOR_LAB2RGB)
cv_result = cv_result * 255

print(cv_result[0][0])

cv.imwrite("img_result_cv.png", cv_result)

[175.0524  229.56917 237.85269]


True